# 1. Data preparation for getting started with LLM Ops

Below are cells to instantiate the vertex ai and bigquery client.
Big query client is then heavily used to extract data from the stack exchange DB and bifurcate the data in evaluation and model tuning based on user's inputs

In [ ]:
from utils.utils_gcp import get_bq_client

In [ ]:
bq_client = get_bq_client()
print("Big query client available")

In [ ]:
QUERY_TABLES = """
SELECT
  table_name
FROM
  `bigquery-public-data.stackoverflow.INFORMATION_SCHEMA.TABLES`
"""

In [ ]:
query_job = bq_client.query(QUERY_TABLES)

In [ ]:
for row in query_job:
    for value in row.values():
        print(value)

In [ ]:
INSPECT_QUERY = """
SELECT
    *
FROM
    `bigquery-public-data.stackoverflow.posts_questions`
LIMIT 3
"""

In [ ]:
import pandas as pd
import pyarrow as pa

In [ ]:
query_job  = bq_client.query(INSPECT_QUERY)

In [ ]:
try:
    stack_overflow_df = query_job\
    .result()\
    .to_arrow()\
    .to_pandas()
    print(stack_overflow_df.head())

except Exception as e:
    print('Exception occurred.', e)

In [ ]:
QUERY_ALL = """
SELECT
    *
FROM
    `bigquery-public-data.stackoverflow.posts_questions` q
"""

In [ ]:
query_job = bq_client.query(QUERY_ALL)

In [ ]:
try:
    stack_overflow_df = query_job\
    .result()\
    .to_arrow()\
    .to_pandas()
except Exception as e:
    print('The DataFrame is too large to load into memory.', e)

In [ ]:
QUERY = """
SELECT
    CONCAT(q.title, q.body) as input_text,
    a.body AS output_text
FROM
    `bigquery-public-data.stackoverflow.posts_questions` q
JOIN
    `bigquery-public-data.stackoverflow.posts_answers` a
ON
    q.accepted_answer_id = a.id
WHERE
    q.accepted_answer_id IS NOT NULL AND
    REGEXP_CONTAINS(q.tags, "python") AND
    a.creation_date >= "2020-01-01"
LIMIT
    10000
"""

In [ ]:
query_job = bq_client.query(QUERY)

In [ ]:
stack_overflow_df = query_job .result().to_arrow().to_pandas()
print(stack_overflow_df.head())

In [ ]:
INSTRUCTION_TEMPLATE = f"""\
Please answer the following Stackoverflow question on Python. Answer it like you are a developer answering Stackoverflow questions.
Stackoverflow question:
"""

In [ ]:
stack_overflow_df['input_text_instruct'] = INSTRUCTION_TEMPLATE + ' '\
    + stack_overflow_df['input_text']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Divide the data into a training and evaluation. By default, 80/20 split is used.
# This (80/20 split) allows for more data to be used for tuning. The evaluation split is used as unseen data during tuning to evaluate performance.
# The random_state parameter is used to ensure random sampling for a fair comparison.
# test_size=0.2 means 20% for evaluation which then makes train set to be of 80%


train, evaluation = train_test_split(
    stack_overflow_df,
    test_size=0.2,
    random_state=42
)

In [ ]:
import datetime

In [ ]:
date = datetime.datetime.now().strftime("%H:%d:%m:%Y")

In [ ]:
cols = ['input_text_instruct','output_text']
tune_jsonl = train[cols].to_json(orient="records", lines=True)
evaluation_jsonl = evaluation[cols].to_json(orient="records", lines=True)

In [ ]:
training_data_filename = f"tune_data_stack_overflow_python_qa.jsonl"
evaluation_data_filename = f"tune_eval_data_stack_overflow_python_qa.jsonl"

In [ ]:
with open(training_data_filename, "w") as f:
    f.write(tune_jsonl)

with open(evaluation_data_filename, "w") as f:
    f.write(evaluation_jsonl)

# 2. Automation and orchestration with pipelines

Below are cells to showcase orchestration and automation of a workflow using [Kubeflow Pipelines](https://www.kubeflow.org/docs/components/pipelines/v2/)

Kubeflow Pipelines is an open source framework.
It's like a construction kit for building machine learning pipelines, making it easy to orchestrate and automate complex tasks.

In [ ]:

import kfp
from kfp import dsl
from kfp import compiler
from kfp.local import SubprocessRunner

import warnings
warnings.filterwarnings("ignore",
                        category=FutureWarning,
                        module='kfp.*')

kfp.local.init(runner=SubprocessRunner())

In [ ]:
@dsl.component
def say_hello(name: str) -> str:
    hello_message = f"Hello, {name}!"
    return hello_message

- Note when passing in values to the a `dsl.component` function, you have to specify the argument names (keyword arguments), and can't use positional arguments.

In [ ]:
hello_task = say_hello(name="vedika")
print(hello_task)

In [ ]:
@dsl.component
def how_are_you(hello_message: str) -> str:
    how_are_you_message = f"How are you? {hello_message}"
    return how_are_you_message

- Notice that when we pass in the return value from the `say_hello` function, we want to pass in the PipelineTask.output object, and not the PipelineTask object itself.

In [ ]:
@dsl.pipeline
def hello_pipeline(recipient: str) -> str:
    hello_task = say_hello(name=recipient)
    how_are_you_task = how_are_you(hello_message=hello_task.output)
    return how_are_you_task.output

##### Implement the pipeline

- A pipeline is a set of components that you orchestrate.
- It lets you define the order of execution and how data flows from one step to another.
- Compile the pipeline into a yaml file, `pipeline.yaml`
- You can look at the `pipeline.yaml` file in your workspace by going to `File --> Open...`. Or right here in the notebook (two cells below)

In [ ]:
compiler.Compiler().compile(hello_pipeline, 'hello_pipeline.yaml')

In [ ]:
pipeline_arguments = {
    "recipient": "World!",
}

In [ ]:
!cat hello_pipeline.yaml

In [ ]:
from google.cloud.aiplatform import PipelineJob
from utils.utils_gcp import init_vertex_ai

init_vertex_ai()
job = PipelineJob(
        ### path of the yaml file to execute
        template_path="hello_pipeline.yaml",
        ### name of the pipeline
        display_name=f"deep_learning_ai_pipeline",
        ### pipeline arguments (inputs)
        ### {"recipient": "World!"} for this example
        parameter_values=pipeline_arguments,
        ### region of execution
        location="us-central1",
        ### root is where temporary files are being
        ### stored by the execution engine
        pipeline_root="./",
)

In [ ]:
job.submit() #submits job for execution

In [ ]:
job.state #check to see the status of the job

In [ ]:
TRAINING_DATA_URI = "./tune_data_stack_overflow_python_qa.jsonl"
EVALUATION_DATA_URI = "./tune_eval_data_stack_overflow_python_qa.jsonl"

- Provide the model with a version.
- Versioning model allows for:
  - Reproducibility: Reproduce your results and ensure your models perform as expected.
  - Auditing: Track changes to your models.
  - Rollbacks: Roll back to a previous version of your model.

In [ ]:
template_path = 'https://us-kfp.pkg.dev/ml-pipeline/large-language-model-pipelines/tune-large-model/v2.0.0'
date = datetime.datetime.now().strftime("%H:%d:%m:%Y")
MODEL_NAME = f"deep-learning-ai-model-{date}"

- This example uses two PaLM model parameters:
  - `TRAINING_STEPS`: Number of training steps to use when tuning the model. For extractive QA you can set it from 100-500.
  - `EVALUATION_INTERVAL`: The interval determines how frequently a trained model is evaluated against the created *evaluation set* to assess its performance and identify issues. Default will be 20, which means after every 20 training steps, the model is evaluated on the evaluation dataset.

In [ ]:
TRAINING_STEPS = 200
EVALUATION_INTERVAL = 20

In [ ]:
pipeline_arguments = {
    "model_display_name": MODEL_NAME,
    "location":  "us-central1",
    "large_model_reference": "text-bison@001",
    "project": "light-river-469808-p4",
    "train_steps": TRAINING_STEPS,
    "dataset_uri": TRAINING_DATA_URI,
    "evaluation_interval": EVALUATION_INTERVAL,
    "evaluation_data_uri": EVALUATION_DATA_URI,
}

pipeline_root = "./"

job = PipelineJob(template_path=template_path,
                  display_name=f"deep_learning_ai_pipeline-{date}",
                    parameter_values=pipeline_arguments,
                  location="us-central1",
                  pipeline_root= pipeline_root,
                  enable_caching= True)

In [ ]:
job.submit()

# L4: Predictions, Prompts and Safety

In [ ]:
import vertexai
from vertexai.language_models import TextGenerationModel
from utils.utils_gcp import init_vertex_ai

In [ ]:
init_vertex_ai()

In [ ]:
model = TextGenerationModel.from_pretrained("text-bison@001")
list_tuned_models = model.list_tuned_model_names()
for i in list_tuned_models:
    print (i)

In [ ]:
import random
tuned_model_select = random.choice(list_tuned_models)

deployed_model = TextGenerationModel.get_tuned_model(tuned_model_select)
PROMPT = "How can I load a csv file using Pandas?"
response = deployed_model.predict(PROMPT)
print(response)

In [ ]:
from pprint import pprint

In [ ]:
output = response._prediction_response[0]
pprint(output)
final_output = response._prediction_response[0][0]["content"]
print(final_output)

#### Prompt Management and Templates
- Remember that the model was trained on data that had an `Instruction` and a `Question` as a `Prompt` (Lesson 2).
- In the example above, *only*  a `Question` as a `Prompt` was used for a response.
- It is important for the production data to be the same as the training data. Difference in data can effect the model performance.
- Add the same `Instruction` as it was used for training data, and combine it with a `Question` to be used as a `Prompt`.

In [ ]:
INSTRUCTION = """\
Please answer the following Stackoverflow question on Python.\
Answer it like\
you are a developer answering Stackoverflow questions.\
Question:
"""

QUESTION = "How can I store my TensorFlow checkpoint on\
Google Cloud Storage? Python example?"

PROMPT = f"""
{INSTRUCTION} {QUESTION}
"""
print(PROMPT)

In [ ]:
final_response = deployed_model.predict(PROMPT)
output = final_response._prediction_response[0][0]["content"]
print(output)

### Safety Attributes
- The reponse also includes safety scores.
- These scores can be used to make sure that the LLM's response is within the boundries of the expected behaviour.
- The first layer for this check, `blocked`, is by the model itself.

In [ ]:
### retrieve the "blocked" key from the
### "safetyAttributes" of the response
blocked = response._prediction_response[0][0]\
['safetyAttributes']['blocked']
print(blocked)

- The second layer of this check can be defined by you, as a practitioner, according to the thresholds you set.
- The response returns probabilities for each safety score category which can be used to design the thresholds.

In [ ]:
### retrieve the "safetyAttributes" of the response
safety_attributes = response._prediction_response[0][0]['safetyAttributes']
pprint(safety_attributes)

### Citations
- Ideally, a LLM should generate as much original cotent as possible.
- The `citationMetadata` can be used to check and reduce the chances of a LLM generating a lot of existing content.

In [ ]:
### retrieve the "citations" key from the
### "citationMetadata" of the response
citation = response._prediction_response[0][0]\
['citationMetadata']['citations']

In [ ]:
pprint(citation)